# Relational Databases with [`SQLachemly`](https://www.sqlalchemy.org)

In [9]:
from sqlalchemy import create_engine, inspect, text
import pandas as pd

In [10]:
# Create DB engine
engine = create_engine('sqlite:///../national_parks_lite.db')
print(f"type; {type(engine)}")

type; <class 'sqlalchemy.engine.base.Engine'>


In [11]:
# Use the inspect function to get table names
inspector = inspect(engine)
tables = inspector.get_table_names()
print(f"tables:\n{tables}")

tables:
['birds', 'mammals', 'parks', 'records']


In [12]:
# Explore columns in each table
for table in tables:
    columns = inspector.get_columns(table)
    print(f"\nTable: {table}")
    for column in columns:
        print(f"  {column['name']} - {column['type']}")


Table: birds
  species_code - TEXT
  order - TEXT
  family - TEXT
  scientific_name - TEXT
  common_names - TEXT
  raptor_group - TEXT
  is_raptor - INTEGER

Table: mammals
  species_code - TEXT
  order - TEXT
  family - TEXT
  scientific_name - TEXT
  common_names - TEXT
  predator_group - TEXT
  is_large_predator - INTEGER

Table: parks
  park_code - TEXT
  park_name - TEXT
  state - TEXT
  square_km - REAL
  latitude - REAL
  longitude - REAL

Table: records
  park_code - TEXT
  species_code - TEXT
  record_status - TEXT
  occurrence - TEXT
  nativeness - TEXT
  abundance - TEXT
  seasonality - TEXT
  conservation_status - TEXT
  is_protected - INTEGER


-----
&nbsp;
### Generate a table of top-10 raptor species by US National Park

In [13]:
# Return a DataFrame corresponding to a SQL query
sql_query = '''SELECT
                    p.park_name,
                    p.state,
                    COUNT(*) AS raptor_count
                FROM records AS r
                LEFT JOIN birds AS b ON r.species_code = b.species_code
                LEFT JOIN parks AS p ON r.park_code = p.park_code
                WHERE b.is_raptor = True
                GROUP BY p.park_name, p.state
                ORDER BY raptor_count DESC
                LIMIT 10;
            '''

In [14]:
df = pd.read_sql_query(sql_query, engine)
df

,park_name,state,raptor_count
0,Death Valley,"CA, NV",43
1,Big Bend,TX,42
2,Redwood,CA,40
3,Grand Canyon,AZ,37
4,Carlsbad Caverns,NM,36
5,Hot Springs,AR,35
6,Saguaro,AZ,34
7,Crater Lake,OR,32
8,Sequoia and Kings Canyon,CA,32
9,Yellowstone,"WY, MT, ID",32


In [18]:
sql_query2 = '''SELECT
    r.park_code AS park_code,
    p.park_name,
    p.state,
    b.scientific_name,
    b.common_names,
    r.occurrence,
    r.nativeness,
    r.abundance,
    r.seasonality,
    r.conservation_status
FROM records AS r
LEFT JOIN birds AS b ON r.species_code = b.species_code
LEFT JOIN parks AS p ON r.park_code = p.park_code
WHERE b.common_names = 'California Condor'
ORDER BY r.occurrence DESC, r.abundance DESC;
'''

In [19]:
df2 = pd.read_sql_query(sql_query2, engine)
df2

,park_code,park_name,state,scientific_name,common_names,occurrence,nativeness,abundance,seasonality,conservation_status
0,ZION,Zion,UT,Gymnogyps californianus,California Condor,Present,Native,Unknown,Vagrant,Endangered
1,ARCH,Arches,UT,Gymnogyps californianus,California Condor,Present,Native,Occasional,Vagrant,Endangered
2,BRCA,Bryce Canyon,UT,Gymnogyps californianus,California Condor,Present,Native,Occasional,Vagrant,Endangered
3,MEVE,Mesa Verde,CO,Gymnogyps californianus,California Condor,Present,Native,Occasional,Vagrant,Endangered
4,GRCA,Grand Canyon,AZ,Gymnogyps californianus,California Condor,Present,Native,Common,Breeder,Endangered
5,PINN,Pinnacles,CA,Gymnogyps californianus,California Condor,Present,Native,Common,Resident,Endangered
6,GRBA,Great Basin,NV,Gymnogyps californianus,California Condor,Not Present (Historical Report),Native,Unknown,Unknown,Endangered
7,LAVO,Lassen Volcanic,CA,Gymnogyps californianus,California Condor,Not Present (Historical Report),Native,Unknown,Unknown,Endangered
8,SEKI,Sequoia and Kings Canyon,CA,Gymnogyps californianus,California Condor,Not Present (Historical Report),Native,Unknown,Unknown,Endangered
9,YOSE,Yosemite,CA,Gymnogyps californianus,California Condor,Not Present (Historical Report),Native,Unknown,Unknown,Endangered
